Updated Fake News Detection Model
Improvements: Full dataset, better preprocessing, ngrams, cross-validation, ensemble methods

In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np

In [ ]:
# Download required NLTK data
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hcpsr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load dataset
df = pd.read_csv("WELFAKE_Dataset.csv")
print(f"Original dataset shape: {df.shape}")

Original dataset shape: (72134, 4)


IMPROVEMENT 1: Using FULL dataset instead of just 5000 samples
Removed the sampling line to use all 72,134 samples for better generalization

In [ ]:
# Handle missing values
df = df.dropna()
print(f"Dataset shape after removing NaN: {df.shape}")

Dataset shape after removing NaN: (71537, 4)


In [ ]:
# IMPROVEMENT 2: Modified text cleaning to preserve some stylistic features
def clean_text(text):
    '''
    Improved cleaning function that preserves important stylistic features
    like excessive punctuation and capitalization patterns
    '''
    text = str(text).lower()

    # Keep count of exclamation and question marks before removing (feature engineering)
    exclamation_count = text.count('!')
    question_count = text.count('?')

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Keep some punctuation patterns but remove excess
    text = re.sub(r'[^a-zA-Z\s!?]', '', text)

    # Tokenization and stemming
    ps = PorterStemmer()
    text = text.split()
    text = [ps.stem(word) for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)

    return text

In [ ]:
# Apply text cleaning
print("Cleaning text data...")
df['text'] = df['text'].apply(clean_text)
df['title'] = df['title'].apply(clean_text)

Cleaning text data...


In [ ]:
# Combine title and text for better context
df['content'] = df['title'] + ' ' + df['text']

In [ ]:
# Prepare features and labels
X = df['content']
y = df['label']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# IMPROVEMENT 3: Enhanced TF-IDF with ngrams and better hyperparameters
print("Vectorizing text with improved TF-IDF...")
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=10000,      # Increased from default 5000
    ngram_range=(1, 2),      # Capture unigrams and bigrams
    min_df=5,                # Ignore terms appearing in less than 5 documents
    max_df=0.7,              # Ignore terms appearing in more than 70% of documents
    sublinear_tf=True        # Apply sublinear scaling (log of term frequency)
)

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
print(f"Training set shape: {X_train_tfidf.shape}")
print(f"Test set shape: {X_test_tfidf.shape}")

IMPROVEMENT 4: Train multiple models with better hyperparameters

In [ ]:
# Logistic Regression with tuned parameters
print("\nTraining Logistic Regression...")
lr = LogisticRegression(max_iter=1000, C=1.0, solver='liblinear', random_state=42)
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_test_tfidf)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {accuracy_lr:.4f}")

In [ ]:
# Random Forest with tuned parameters
print("\nTraining Random Forest...")
rf = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=42, n_jobs=-1)
rf.fit(X_train_tfidf, y_train)
y_pred_rf = rf.predict(X_test_tfidf)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")

In [ ]:
# SVC with probability enabled and linear kernel
print("\nTraining SVC...")
svc = SVC(kernel='linear', C=1.0, probability=True, random_state=42)
svc.fit(X_train_tfidf, y_train)
y_pred_svc = svc.predict(X_test_tfidf)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"SVC Accuracy: {accuracy_svc:.4f}")

In [ ]:
# IMPROVEMENT 5: Ensemble model using voting classifier
print("\nTraining Ensemble Model...")
ensemble = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('svc', svc)
    ],
    voting='soft',  # Use probability-based voting
    weights=[1, 1, 1]
)
ensemble.fit(X_train_tfidf, y_train)
y_pred_ensemble = ensemble.predict(X_test_tfidf)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print(f"Ensemble Model Accuracy: {accuracy_ensemble:.4f}")

In [ ]:
# IMPROVEMENT 6: Cross-validation to check generalization
print("\nPerforming 5-Fold Cross-Validation on Ensemble...")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X_full_tfidf = tfidf.fit_transform(X)
cv_scores = cross_val_score(ensemble, X_full_tfidf, y, cv=skf, scoring='accuracy', n_jobs=-1)
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

In [ ]:
# Detailed evaluation of best model (Ensemble)
print("\n" + "="*50)
print("ENSEMBLE MODEL - DETAILED EVALUATION")
print("="*50)
print("\nClassification Report:")
print(classification_report(y_test, y_pred_ensemble, target_names=['Real', 'Fake']))

In [ ]:
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred_ensemble)
print(cm)
print(f"\nTrue Negatives (Real predicted as Real): {cm[0][0]}")
print(f"False Positives (Real predicted as Fake): {cm[0][1]}")
print(f"False Negatives (Fake predicted as Real): {cm[1][0]}")
print(f"True Positives (Fake predicted as Fake): {cm[1][1]}")

In [ ]:
# Save the best model and vectorizer
import pickle

In [ ]:
print("\nSaving ensemble model and vectorizer...")
pickle.dump(ensemble, open('ensemble_model.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf_vectorizer.pkl', 'wb'))
print("Models saved successfully!")

In [ ]:
# IMPROVEMENT 7: Function to test on real-world news
def predict_news(news_text):
    '''
    Function to predict if a news article is fake or real
    '''
    cleaned = clean_text(news_text)
    vectorized = tfidf.transform([cleaned])
    prediction = ensemble.predict(vectorized)[0]
    probability = ensemble.predict_proba(vectorized)[0]

    label = "FAKE" if prediction == 1 else "REAL"
    confidence = probability[prediction] * 100

    print(f"\nPrediction: {label}")
    print(f"Confidence: {confidence:.2f}%")
    print(f"Probability - Real: {probability[0]:.4f}, Fake: {probability[1]:.4f}")

    return label, confidence

In [ ]:
# Test with a sample
print("\n" + "="*50)
print("TESTING WITH SAMPLE NEWS")
print("="*50)
sample_news = "BREAKING: Scientists discover cure for all diseases overnight!"
print(f"Sample: {sample_news}")
predict_news(sample_news)

In [ ]:
print("\n✅ Model training complete! Key improvements:")
print("1. Using full dataset (72K+ samples instead of 5K)")
print("2. Better text preprocessing preserving stylistic features")
print("3. TF-IDF with bigrams for phrase-level patterns")
print("4. Ensemble model combining 3 algorithms")
print("5. Cross-validation for better generalization assessment")
print("6. Probability outputs for confidence scores")
print("\nTo use: Call predict_news('your news text here')")